# ТВиМС Задание 4

## Вариант 14

Проверка статистических гипотез.
Критерии согласия Пирсона, Колмогорова, Мизеса.

In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]

In [3]:
a = 0
b = math.pi / 4

In [4]:
def get_x():
    return  + random.random()*(b-a)

def y_func(x):
    return math.tan(x)

def get_y():
    return y_func(get_x())

In [5]:
def generate_varseries(n):
    ys = []

    for i in range(n):
        ys.append(get_y())

    ys.sort()
    return ys

In [6]:
def Fy_deduced(y):
    if y < 0:
        return 0
    elif y > 1:
        return 1
    else:
        return 4 / math.pi * math.atan(y)

def fy_deduced(y):
    if y < 0 or y > 1:
        return 0
    else:
        return 4 / (math.pi * (1 + y**2))

***

In [7]:
# (copied from second_and_third.ipynb with slight modifications)
def histogram_data_beta(a, b, vs, group_num, normalize=True):
    """Return data needed for type 2 histogram
    as a list of tuples each being:
    (a bin's left bound, a bin's occurrences count)
    
    Each bin's interval has its left bound included
    and right excluded, e.g. [0, 1), [1, 2), [3, 4), ...

    Примечание: работает только для НСВ с почти-лишь-только-один-раз-встречающимися
    элементами вариационного ряда (здесь вар-ряд = выборка из-за *высокой степени
    уникальности* элементов выборки, т.е. элементы списка 'vs' могут повторяться,
    но это очень маловероятно, если число b-a неотрицательного порядка/
    однозначного отрицательного порядка - порядок условно от -9 до 10+)
    """
    bin_capacity = len(vs) // group_num

    hdata = [[a, 0]]
    for i in range(len(vs)):
        hdata[-1][1] += 1
        last_bin_empty = False

        # if the last bin is full:
        if hdata[-1][1] == bin_capacity:
            # calculate its right bound:
            if i == len(vs) - 1:
                right_bound = b
            else:
                right_bound = (vs[i] + vs[i+1]) / 2

            # add a new empty bin:
            hdata.append([right_bound, 0])
            last_bin_empty = True

    if last_bin_empty:
        del hdata[-1]

    if normalize:
        square = 0
        for i in range(len(hdata) - 1):
            # calculating local density and updating square:
            hdata[i][1] = hdata[i][1] / (hdata[i+1][0] - hdata[i][0])
            square += hdata[i][1] * (hdata[i+1][0] - hdata[i][0])
        
        hdata[-1][1] = hdata[-1][1] / (b - hdata[-1][0])
        square += hdata[-1][1] * (b - hdata[-1][0])

        for hd in hdata:
            hd[1] /= square
    else:
        for hd in hdata:
            hd[1] /= len(vs)

    hdata.append([b, hdata[-1][1]])
            
    return hdata

In [8]:
# plotting histogram beta (copied from second_and_third.ipynb)
def plot_histogram_beta_and_polygon_and_fy(a, b, n, group_num, normalize=True):
    vs = generate_varseries(n)
    hdata = histogram_data_beta(a, b, vs, group_num, normalize)
    ps = [(hdata[i][0] + hdata[i+1][0]) / 2 for i in range(len(hdata) - 1)]

    # plotting histogram:
    plt.step([hd[0] for hd in hdata],
             [hd[1] for hd in hdata],
             where='post')
    
    # plotting polygon:
    plt.scatter(ps,
                [hd[1] for hd in hdata[:-1]])
    plt.plot(ps,
             [hd[1] for hd in hdata[:-1]],
             label='полигон распределения')

    # plotting analytic:
    xs = np.linspace(a, b, 1000)
    plt.plot(xs,
             [fy_deduced(x) for x in xs],
             label='аналитическая функция плотности')

    plt.legend()
    plt.show()

***

### Критерий согласия Пирсона

In [9]:
def xi_squared_criteria(a, b, n, group_num, f_hypothesis):
    vs = generate_varseries(n)
    hdata = histogram_data_beta(a, b, vs, group_num, normalize=False)

    hypothesis_ys = [Fy_deduced(hdata[i+1][0]) - Fy_deduced(hdata[i][0])
                     for i in range(group_num)]
    histogram_ys = [hd[1] for hd in hdata[:-1]]

    # checking the condition:
    # assert abs(1 - sum(hypothesis_ys)) <= 0.01

    summands = []
    for i in range(group_num):
        summand = ((histogram_ys[i] - hypothesis_ys[i])**2
                   / (hypothesis_ys[i]))
        summands.append(summand)

    xi_squared = n*sum(summands)
    return xi_squared

In [10]:
# test xi_squared 20 times

for i in range(20):
    xi = xi_squared_criteria(0, 1, 200, 16, fy_deduced)
    print(xi)

0.05703024893966979
12.462342483864251
0.02612977184648224
17.037422621450084
0.037669259411235556
16.905030029471916
0.037457349383090865
9.582068891932932
0.053126465461681316
13.843765562467105
0.038672725749551384
20.28927589575924
0.030873293437593263
11.375500769892513
0.03562061220597368
29.21524163690558
0.04067354329222339
14.003212869969078
0.048539558321992415
12.067028982283801
0.05104566362084184
7.268878987971209
0.04665237934169775
16.50363653509717
0.040245561438695954
9.66404738793056
0.04655760853331059
17.809598551274835
0.05275271461240583
9.27321974231645
0.020487467540579618
14.037200636003805
0.04705588283598605
15.50558758230488
0.027880403480073146
7.671374308572039
0.062105375034691046
16.568371936235238
0.040212588914818825
11.732062238176031


***

### Критерий согласия Колмогорова

***

### Критерий согласия Мизеса